## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

ModuleNotFoundError: No module named 'mingpt'

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
%%writefile dataset.py

import math
import torch
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [ ]:
block_size = 128 # spatial extent of the model for its context

In [ ]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

In [ ]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
from dataset import CharDataset
text = open('input.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

In [ ]:
chars = sorted(list(set(text)))

In [ ]:
''.join(chars)

In [ ]:
vocab_size = len(chars)

In [ ]:
stoi = {ch:i for i,ch in enumerate(chars)}

In [ ]:
itos = {i:ch for i,ch in enumerate(chars)}

In [ ]:
def encode(st):
    return [stoi[c] for c in st]

In [ ]:
encode = lambda s: [stoi[c] for c in s]

In [ ]:
decode = lambda idx:  ''.join([ itos[i] for i in idx])

In [ ]:
print(encode('hi there'))

In [ ]:
print(decode(encode('hi there')))

In [ ]:
import torch

In [ ]:
data = torch.tensor(encode(text), dtype = torch.long)

In [ ]:
data

In [ ]:
data.shape

In [ ]:
n = int(0.9*len(data))

In [ ]:
train_data = data[:n]

In [ ]:
val_data = data[n:]

In [ ]:
block_size = 8

In [ ]:
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]

In [ ]:
y = train_data[1:block_size+1]

In [ ]:
x

In [ ]:
y

In [ ]:
for t in range(block_size):
    context = x[0: t+1]
    target = y[t]
    print("context", context,"target",target)

In [ ]:
#paralellation

In [ ]:
torch.manual_seed(1337)

In [ ]:
batch_size = 4 # how many indepedent sequence will we process in parallel?

In [ ]:
block_size = 8 # what is the maximum context length for prediction? 

In [ ]:
def get_batch(split):
    # generate a small batch of data of input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
xb, yb = get_batch('train')

In [ ]:
xb

In [ ]:
yb

In [ ]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print('context', context, 'target', target)

In [ ]:
vocab_size

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [ ]:
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly read off logits for next token from a look up table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            #print(logits.shape)
            targets = targets.reshape(B*T)
            #print(targets.shape)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            #print('logits', logits.shape)
            logits = logits[:, -1, :] # (B, C)
            #print('logits shape after', logits.shape)
            #print('last tokens logits in generate', logits)
            probs = F.softmax(logits, dim = 1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
            
    

In [ ]:
yb

In [ ]:
m = BigramLM(vocab_size)
logits, loss = m(xb, yb)

In [ ]:
idx = torch.zeros((1,1), dtype = torch.long)

In [ ]:
decode(m.generate(idx, max_new_tokens = 100)[0].tolist())

In [ ]:
print(logits)

In [ ]:
loss

In [ ]:
print(out.shape)

In [ ]:
torch.randint

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [ ]:
batch_size = 32

for steps in range(10000):
    # sample batch of data
    xb, yb = get_batch('train')

    #eval the loss

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
    
    

In [ ]:
decode(m.generate(idx, max_new_tokens = 300)[0].tolist())

In [ ]:
get_batch('train')

In [ ]:
text

In [ ]:
chars

In [ ]:
itos = {i:item for i,item in enumerate(chars)}

In [ ]:
stoi = {item:i for i,item in enumerate(chars)}

In [ ]:
itos

In [ ]:
print(train_dataset.data)

In [ ]:
print(train_dataset.data)

In [ ]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

In [ ]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

In [ ]:
# alright, let's sample some character-level Shakespeare
from mingpt.utils import sample

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

In [ ]:
# well that was fun